In [1]:
import datetime, urllib, re, math, json, requests,io
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import scipy as sp
from scipy import optimize
from sklearn.linear_model import LinearRegression
from IPython.display import display, HTML
from ipywidgets import *


try:
    from packaging import version
except ImportError:
    !pip install packaging

if( version.parse(pd.__version__) < version.parse("0.23.4")):
    print("update pandas")
    !pip install pandas --upgrade #--ignore-installed
    print(version.parse(pd.__version__))

In [75]:
#** pandas
# change max number of rows to show
pd.set_option('display.max_rows', 300)

#** plotters formats

import matplotlib.ticker as mticker
import matplotlib.dates as mdates
from cycler import cycler
#* dpi size
plt.rcParams['figure.dpi'] = 200
myFmt = mdates.DateFormatter('%m/%d')
myLocator = mticker.MultipleLocator(7)

default_cycler = (cycler(marker=['.','*','+','x','s']) *
                  cycler(color=['b','g','m','y','c','r']) *
                  cycler(linestyle=['--']) *
                  cycler(lw=[1]))

plt.rc('axes', prop_cycle=default_cycler)

#** panda data frame styles
from IPython.display import display_html
def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)


In [3]:
df = pd.read_csv("https://coronadatascraper.com/timeseries-tidy.csv")
df.fillna("-1",inplace=True);
dataTCS =df.set_index(["city","county","state","country","type"])
dataTCS["date"] = [datetime.datetime.strptime(ts,"%Y-%m-%d").date() for ts in dataTCS["date"]]



C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [133]:
def plotter(minCount=5000,minCountKpi="cases",equalize=True,equalizeTrg="ITA",equalizeCount=100, plotKpi="deaths",logyPlot=True,
           countryStateSweep=-1, reference=False):
    idx = pd.IndexSlice
        
    fig,ax = plt.subplots(1,1)
    if(equalize):
        if(countryStateSweep==-1):
            g = dataTCS.loc[idx["-1","-1","-1",equalizeTrg,minCountKpi]]
        else:
            g = dataTCS.loc[idx["-1","-1",equalizeTrg,countryStateSweep,minCountKpi]]
        gdts = g[g["value"]>equalizeCount]["date"].values[0]
        gdte = g["date"].values[-1]
        N = (gdte-gdts).days#sp.size(g[g["value"]>equalizeCount]["date"].values)
        print("Equalize to %s : date:%s to %s" % (equalizeTrg,gdts,gdte))
    if(reference):
        for rate in np.array([8., 13., 21., 34.])/100.:            
            d = g[g["date"]>=gdts]["date"]
            #y0= g[g["date"]>=gdts]["value"].values[0]
            y0=equalizeCount
            M= sp.size(d.values)
            ax.plot(d,y0*np.exp(rate * np.arange(0,M)),'k--',label="ref%.1f"%(rate*100))
    if(countryStateSweep==-1):
        idxScreen = lambda ctr :idx["-1","-1","-1",ctr,minCountKpi];
        idxPlot   = lambda ctr :idx["-1","-1","-1",ctr,plotKpi];
        countries = dataTCS.index.levels[3]
    else:
        #ctr becomes a loop variable for state
        idxScreen = lambda ctr :idx["-1","-1",ctr,countryStateSweep,minCountKpi];
        idxPlot   = lambda ctr :idx["-1","-1",ctr,countryStateSweep,plotKpi];
        # using states here
        countries = np.sort(np.unique(dataTCS.loc[idx["-1","-1",:,countryStateSweep,"cases"]].index.values))[1:]
    
    # temp commented out    
    for ctr in countries:    
        if dataTCS.loc[idxScreen(ctr)].value[-1]< minCount: continue     
        try:
            g = dataTCS.loc[idxPlot(ctr),["date","value"]]        
        except:
            continue
        if(equalize):
            #check if sufficent samples exist to equlize
            if False == (g["value"]>equalizeCount).any():continue
            dts = g[g["value"]>equalizeCount]["date"].values[0]
            print("%s --> %d" %(ctr,(dts-gdts).days))
            data= g[g["date"]>=dts]
            M = min(N,sp.size(data["value"]))
            ts = [(gdts + datetime.timedelta(i)) for i in range(0,sp.size(data["value"]))] 
        else:
            ts=g.index
            data=g["value"]
            M=sp.size(g)
        ax.plot(ts[0:M],data["value"][0:M].values,label=ctr)    
    
    ax.xaxis.set_major_locator(myLocator)
    ax.xaxis.set_major_formatter(myFmt)
    ax.grid()
    mm = max(dataTCS.loc[idx[:,:,:,:,plotKpi],"value"])
    ax.set_ylim(equalizeCount * .75, mm * 1.25)
    ax.set_xlim(gdts,gdte)
    ax.legend()
    fig.autofmt_xdate()    
    #plt.title("%s vs time"%plotKpi);

    if(logyPlot) : plt.yscale('log')
        
# country screen
minCount     = 10000
minCountKpi  = "cases" # 'active', 'cases', 'deaths', 'growthFactor', 'recovered', 'tested'
# Equalize (kpi same as plot)
equalize     = True
equalizeTrg  = "ITA"
equalizeCount= 500

# plot 
plotKpi      = "cases"
logyPlot     = True
#plotter(minCount,minCountKpi,equalize,equalizeTrg,equalizeCount, plotKpi,logyPlot)        

In [132]:
from ipywidgets import *


interact(plotter,minCount=(0,1e6,1e4),
         minCountKpi=dataTCS.index.levels[4].unique().values,
#         equalize=[True, False],
         equalizeTrg=dataTCS.index.levels[3].unique().values,
         equalizeCount=(0,10000),
         plotKpi=dataTCS.index.levels[4].unique().values,
#         logyPlot=[True, False],
#         reference=[True,False]
        );#countryStateSweep=dataTCS.index.levels[3].unique().values)

interactive(children=(FloatSlider(value=5000.0, description='minCount', max=1000000.0, step=10000.0), Dropdown…

In [134]:
# states
countryStateSweep="USA"
idx = pd.IndexSlice

interact(plotter,minCount=(0,1e5),
         minCountKpi=dataTCS.index.levels[4].unique().values,
         #equalize=[True, False],
         equalizeTrg=np.sort(np.unique(dataTCS.loc[idx["-1","-1",:,countryStateSweep,"cases"]].index.values))[1:],
         equalizeCount=(0,10000),
         plotKpi=dataTCS.index.levels[4].unique().values,
         #logyPlot=[True, False],
         countryStateSweep=countryStateSweep
        );


interactive(children=(FloatSlider(value=5000.0, description='minCount', max=100000.0), Dropdown(description='m…

In [ ]:
import seaborn as sns
# gradient across two colols, second is one color
cm = sns.diverging_palette(150,10, n=12,l=55,center='light',as_cmap=True)
cm = sns.light_palette('red',as_cmap=True)


dt = datetime.datetime.today().date()-datetime.timedelta(days=1)
dt2 = datetime.datetime.today().date()-datetime.timedelta(days=2)

# dataTCS = dataTCS.astype({"value":"int32"})
# display(dataTCS.loc[idx,"value"][dataTCS.loc[idx,"date"]==dt].to_frame().style
#     .background_gradient(cmap=cm,subset=['value']))

df = pd.DataFrame()
idx = pd.IndexSlice["-1","-1",:,"USA",["cases"]]
df["cases"] = dataTCS.loc[idx,"value"][dataTCS.loc[idx,"date"]==dt].copy()
idx = pd.IndexSlice["-1","-1",:,"USA",:]
#df["deaths"]= dataTCS.loc[idx,"value"][dataTCS.loc[idx,"date"]==dt].values
#df = df.reset_index().drop(["city","county","country","type"],axis=1)#
#df = df[ df["state"] != "-1"]

df = dataTCS.reset_index().copy()
g1 = df[(df["type"]=="cases")  & (df["country"]=="USA") & (df["date"]==dt) &  (df["county"]=="-1") & (df["city"]=="-1") & (df["state"]!="-1")].set_index(["state"])["value"].to_frame().rename(columns={"value":'cases1'})
g2 = df[(df["type"]=="deaths") & (df["country"]=="USA") & (df["date"]==dt) &  (df["county"]=="-1") & (df["city"]=="-1") & (df["state"]!="-1")].set_index(["state"])["value"].to_frame().rename(columns={"value":'deaths1'})
g3 = df[(df["type"]=="cases")  & (df["country"]=="USA") & (df["date"]==dt2) & (df["county"]=="-1") & (df["city"]=="-1") & (df["state"]!="-1")].set_index(["state"])["value"].to_frame().rename(columns={"value":'cases2'})
g4 = df[(df["type"]=="deaths") & (df["country"]=="USA") & (df["date"]==dt2) & (df["county"]=="-1") & (df["city"]=="-1") & (df["state"]!="-1")].set_index(["state"])["value"].to_frame().rename(columns={"value":'deaths2'})

midx = pd.MultiIndex.from_tuples([
    (dt,"cases"),(dt,"deaths"),(dt2,"cases"),(dt2,"deaths"),("delta","cases"),("delta","death"),("rate","cases"),("rate","death")
    ])



midx
idx = pd.IndexSlice["-1","-1",:,"USA",["cases"]]
dfFinal = pd.concat([g1,g2,g3,g4], axis=1,sort=False)


dfFinal["casesD"] = dfFinal["cases1"]-dfFinal["cases2"]
dfFinal["deathD"] = dfFinal["deaths1"]-dfFinal["deaths2"] 
dfFinal = dfFinal.replace(0,-1)
dfFinal["rateC"]  = (dfFinal["cases1"]-dfFinal["cases2"])  /dfFinal["cases2"] 
dfFinal["rateD"]  = (dfFinal["deaths1"]-dfFinal["deaths2"])/dfFinal["deaths2"] 
dfFinal.columns=midx
display(dfFinal.sort_values(by=("delta","cases"),ascending=False).head(12)
    .style
    .background_gradient(cmap=cm, subset=['delta',"rate"])
    .format({('rate',"cases"): "{:.1%}",('rate',"death"): "{:.1%}"})
    )

